### 1. Imports and Global Parameters
Import necessary libraries and define the core parameters for the simulation environment and vehicles.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.transforms import Affine2D
from IPython.display import display, clear_output
import math

# Simulation Parameters
dt = 0.1  # Time step (s)

# Roundabout Geometry
inner_radius = 46  # Inner radius of the roundabout (m)
outer_radius = 84  # Outer radius of the roundabout (m)

# Vehicle Properties
vehicle_width = 1.8  # Vehicle width (m)
vehicle_length = 4.5  # Vehicle length (m)

# IDM (Intelligent Driver Model) Parameters
desired_speed = 15    # Desired speed (m/s)
max_acceleration = 3    # Maximum acceleration (m/s^2)
comfortable_deceleration = 4.0  # Comfortable deceleration (m/s^2)
s0 = 4.0              # Minimum safe distance / jam distance (m)
T = 1.0               # Desired time headway (s)

### 2. Traffic Flow Models and Vehicle Class
This section contains the core logic for the simulation.
- **Car-Following Models (`idm_acceleration`, `IDM`: The function name is IDM, but it actually represents Tangential IAM.)**: Implement the Intelligent Driver Model to manage tangential speed based on surrounding vehicles.
- **Improve the intelligent agent model (`IAM`)**: Two-dimensional movements in both radial and tangential directions have been achieved..
- **Helper Functions**: Functions to find leading and following vehicles.
- **`Vehicle` Class**: Defines the state and behavior of each vehicle in the simulation.

In [19]:
def idm_acceleration(v, v_lead, gap, sy):
    """IDM calculation for tangential acceleration with lateral influence."""
    def alphalongfun(sy):
        return min(1, np.exp(-(abs(sy) - 4.846) / 0.6))
    
    delta_v = v - v_lead
    s_star = s0 + max(0, T * v + v * delta_v / (2 * np.sqrt(max_acceleration * comfortable_deceleration)))
    acc = max_acceleration * (1 - (v / desired_speed) ** 4) + np.maximum(-12, -alphalongfun(sy) * max_acceleration * (s_star / gap) ** 2)
    return acc

def idm_acceleration1(v, v_lead, gap, sy):
    """Variant of IDM for specific scenarios (e.g., approaching exit)."""
    def alphalongfun(sy):
        return min(1, np.exp(-(abs(sy) - 2) / 0.6))
    
    delta_v = v - v_lead
    s_star = max(0, T * v + v * delta_v / (2 * np.sqrt(max_acceleration * comfortable_deceleration)))
    acc = max_acceleration * (1 - (v / desired_speed) ** 4) + np.maximum(-4, -alphalongfun(sy) * max_acceleration * (s_star / gap) ** 2)
    if v < 1:
        acc = 0.1
    return acc

def IDM(v, v_lead, gap):
    """A simplified IDM for vehicles entering the roundabout."""
    delta_v = v - v_lead
    s_star = 1 + max(0, 0.5 * v + v * delta_v / (4 * np.sqrt(3 * comfortable_deceleration)))
    acc = 5 * (1 - (v / 15) ** 4) + np.maximum(-18, -3 * (s_star / gap) ** 2)
    return acc

def idm_int(v, v_lead, gap, sy):
    """Calculates the interaction term of the IDM, representing deceleration due to a front vehicle."""
    def alphalongfun(sy):
        return min(1, np.exp(-(abs(sy) - 4.846) / 0.6))
    
    delta_v = v - v_lead
    s_star = s0 + max(0, T * v + v * delta_v / (2 * np.sqrt(max_acceleration * comfortable_deceleration)))
    acc = -(max_acceleration * (s_star / gap) ** 2) * alphalongfun(sy)
    return acc

def IAM(a, A, B, C, D, ego_vehicle, front_vehicle):
    """IAM calculation for radial (lane-changing) acceleration."""
    Wveh, Wl = ego_vehicle['width'], front_vehicle['width']
    vy = ego_vehicle['speed_y']
    vy1 = front_vehicle['speed_y']
    dy = front_vehicle['position_y'] - ego_vehicle['position_y']
    sign_dy = -1 if dy < 0 else 1
    Wavg = ego_vehicle['width']
    overlap = abs(dy) < Wavg
    alpha = sign_dy * (abs(dy) / Wavg if overlap else np.exp(-(abs(dy) - Wavg) / B))
    v0LatInt = A * alpha * (a - max_acceleration * (1 - (ego_vehicle['speed_x'] / desired_speed) ** 4))
    mult_dv_factor = 1 if overlap else max(0.0, 1.0 - C * sign_dy * (vy1 - vy))
    accLatInt = (v0LatInt) / D * mult_dv_factor
    return max(-2, min(2, accLatInt - vy / D))

def calculate_angle_gap(current_angle, front_angle):
    """Calculate the shortest positive angle from current_angle to front_angle."""
    gap_angle = (front_angle - current_angle) % (2 * np.pi)
    return gap_angle

class Vehicle:
    def __init__(self, idx, angle, radius, entry_angle, exit_angle):
        self.idx = idx
        self.angle = angle
        self.radius = radius
        self.entry_angle = entry_angle % (2 * np.pi)
        self.exit_angle = exit_angle % (2 * np.pi)
        self.tangential_speed = 0
        self.radial_speed = -10
        self.tangential_acc = 0
        self.radial_acc = 0
        self.width = vehicle_width
        self.length = vehicle_length
        self.position_x = []
        self.position_y = []
        self.tangential_speeds = []
        self.radial_speeds = []
        self.in_roundabout = False
        self.paused = True
        self.entry_idx = None
        self.exit_idx = None
        self.decide = 100  # Decision state
        self.gammar = 3    # Parameter influencing lane change behavior
        self.max_angle = np.deg2rad(60)
        self.desired_speed = 15
        self.T = 1
        self.out = False

    def update_decision(self, entry_angles, exit_angles):
        """Updates the vehicle's decision state (enter or exit)."""
        if self.decide < 0:
            return self.decide
            
        if self.radius <= outer_radius:
            self.angle = self.angle % (2 * np.pi)
            
            # This complex set of conditions determines if the vehicle should focus on entering or exiting.
            # It compares the vehicle's current angle to various ranges defined by its entry and exit points.
            entry_range_start = entry_angles[self.entry_idx] % (2 * np.pi)
            if calculate_angle_gap(self.entry_angle, self.exit_angle) < np.pi / 2:
                entry_range_end = (entry_angles[self.entry_idx] + np.pi / 2) % (2 * np.pi)
            else:
                Cita = calculate_angle_gap(self.entry_angle, self.exit_angle) / 2
                entry_range_end = (entry_angles[self.entry_idx] + Cita) % (2 * np.pi)
                
            exit_range_end = exit_angles[self.exit_idx] % (2 * np.pi)
            
            # There are multiple relationship checks here that could likely be simplified.
            # The core idea is to set `self.decide` to a positive value for the entry phase
            # and a negative value for the exit phase.
            if entry_range_start <= entry_range_end:
                if entry_range_start <= self.angle < entry_range_end:
                    self.decide = self.entry_idx + 1  # Entry Phase
                else:
                    self.decide = -(self.exit_idx + 1) # Exit Phase
            else: # Wraps around 2*pi
                if self.angle >= entry_range_start or self.angle < entry_range_end:
                    self.decide = self.entry_idx + 1 # Entry Phase
                else:
                    self.decide = -(self.exit_idx + 1) # Exit Phase
        return self.decide

    def update(self, front_vehicle, follower_vehicle, dt):
        """Updates vehicle state for one time step."""
        def H(x):
            return np.where(x >= 0, 1, 0)

        if self.radius >= outer_radius:
            proximity_to_max = 0
        elif self.radius <= inner_radius:
            proximity_to_max = 1
        else:
            proximity_to_max = (outer_radius - self.radius) / (outer_radius - inner_radius)
        
        T = self.T - 0.2 * proximity_to_max
        desired_speed_local = self.desired_speed + 5 * proximity_to_max

        effective_inner_radius, effective_outer_radius = self.calculate_effective_radius()

        if self.radius < outer_radius:
            if front_vehicle == 1: # Special condition for approaching exit
                angle_diff_exit = calculate_angle_gap(self.angle, (self.exit_angle + np.pi/36) % (2 * np.pi))
                gap_to_exit = self.radius * abs(angle_diff_exit)
                sy1 = outer_radius - self.radius
                self.tangential_acc = idm_acceleration1(self.tangential_speed, 0, gap_to_exit, sy1)
                
                angle_diff11 = calculate_angle_gap(self.angle, (self.exit_angle-np.pi/36)% (2 * np.pi))
                gap1 = self.radius * abs(angle_diff11)
                new_radial_speed = self.tangential_speed / gap1 * sy1
                self.radial_acc = (new_radial_speed - self.radial_speed) / dt + self.targetF() + self.calcAccB(effective_outer_radius, effective_inner_radius, self.radius)

            elif front_vehicle is None:
                tangential_acc = max_acceleration * (1 - (self.tangential_speed / desired_speed_local) ** 4)
                if follower_vehicle:
                    angle_diff = calculate_angle_gap(follower_vehicle.angle, self.angle)
                    gap = min(self.radius * abs(angle_diff), follower_vehicle.radius * abs(angle_diff)) - self.length
                    sy = follower_vehicle.radius - self.radius
                    self.tangential_acc = tangential_acc - max(-2, 0.6 * idm_acceleration(follower_vehicle.tangential_speed, self.tangential_speed, gap, sy))
                else:
                    self.tangential_acc = tangential_acc
                self.radial_acc = self.targetF() * H(self.tangential_acc) + self.calcAccB(effective_outer_radius, effective_inner_radius, self.radius)

            else: # Standard following behavior
                angle_diff = calculate_angle_gap(self.angle, front_vehicle.angle)
                gap = min(self.radius * abs(angle_diff), front_vehicle.radius * abs(angle_diff)) - self.length
                sy = self.radius - front_vehicle.radius
                tangential_acc = idm_acceleration(self.tangential_speed, front_vehicle.tangential_speed, gap, sy)

                if follower_vehicle:
                    angle_diff_f = calculate_angle_gap(follower_vehicle.angle, self.angle)
                    gap_f = min(self.radius * abs(angle_diff_f), follower_vehicle.radius * abs(angle_diff_f)) - self.length
                    sy_f = follower_vehicle.radius - self.radius
                    self.tangential_acc = tangential_acc - max(-2, 0.6 * idm_acceleration(follower_vehicle.tangential_speed, self.tangential_speed, gap_f, sy_f))
                else:
                    self.tangential_acc = tangential_acc

                self.radial_acc = IAM(
                    self.tangential_acc, 1, 0.6, 0.7, 0.5,
                    {'width': self.length, 'speed_y': self.radial_speed, 'position_y': self.radius, 'desired_speed': desired_speed_local, 'speed_x': self.tangential_speed},
                    {'width': front_vehicle.length, 'speed_y': front_vehicle.radial_speed, 'position_y': front_vehicle.radius}
                ) + self.targetF() * H(self.tangential_acc) + self.calcAccB(effective_outer_radius, effective_inner_radius, self.radius)

            # Update kinematics
            radius1 = self.radius + self.radial_speed * dt + self.radial_acc * dt * dt / 2
            self.angle = (self.angle + max(0, self.tangential_speed / max(self.radius, radius1) * dt + self.tangential_acc * dt * dt / 2 / max(self.radius, radius1))) % (2 * np.pi)
            self.radius += self.radial_speed * dt + self.radial_acc * dt**2 / 2
            self.radius = max(inner_radius + self.width / 2, min(self.radius, outer_radius - self.width / 2))
            self.tangential_speed = max(0, self.tangential_speed + self.tangential_acc * dt)
            self.radial_speed += self.radial_acc * dt

            # Constrain vehicle's movement angle
            if self.tangential_speed != 0:
                angle_between = np.arctan(self.radial_speed / self.tangential_speed)
            else:
                angle_between = np.pi / 2
            
            if calculate_angle_gap(self.angle, self.exit_angle) < np.math.asin(20 / outer_radius):
                self.max_angle = np.deg2rad(75)
            else:
                self.max_angle = np.deg2rad(50)
            
            if abs(angle_between) >= self.max_angle:
                new_radial_speed = self.tangential_speed * np.tan(self.max_angle)
                self.radial_speed = np.sign(self.radial_speed) * new_radial_speed

            # Finalizing exit
            angle_diff_to_exit = calculate_angle_gap(self.angle, self.exit_angle)
            if (angle_diff_to_exit < np.math.asin(5 / outer_radius)) and self.decide < 0 and self.radius > outer_radius - 20:
                self.angle = self.exit_angle
                self.radial_speed = 10
                self.radius = outer_radius + 0.1
                self.tangential_speed = 0
                self.tangential_acc = 0
                self.radial_acc = 4/3
                self.out = True
        
        else: # Vehicle is outside the main roundabout area
            if self.decide > 0: # Entering
                if front_vehicle:
                    gap = self.radius - front_vehicle.radius - self.length
                    radial_acc1 = IDM(abs(self.radial_speed), abs(front_vehicle.radial_speed), gap)
                else:
                    radial_acc1 = 5 * (1 - (abs(self.radial_speed) / 20) ** 4)
                self.radial_acc = -radial_acc1
                self.tangential_acc = 0
                self.tangential_speed = 0
                self.radial_speed = min(self.radial_speed + self.radial_acc * dt, 0)
                self.radius += min(self.radial_speed * dt, 0)

            elif self.decide < 0 and self.out:
                self.angle = self.exit_angle
                self.radial_acc = 4/3
                self.radial_speed += self.radial_acc * dt
                self.radius += self.radial_speed * dt
                if self.radial_speed < 0: # Vehicle has fully exited
                    self.radius = 999
            else:
                self.radius = outer_radius - self.width / 2
                self.radial_speeds = 0

        # Record history
        self.position_x.append(self.radius * np.cos(self.angle))
        self.position_y.append(self.radius * np.sin(self.angle))
        self.tangential_speeds.append(self.tangential_speed)
        self.radial_speeds.append(self.radial_speed)

    def targetF(self):
        """Calculates a force directing the vehicle towards its goal (inner lane or exit lane)."""
        position_x_local = self.radius * calculate_angle_gap(self.entry_angle, self.angle)
        position_y_local = self.radius

        if self.decide > 0: # Entering
            target_y = inner_radius + self.length + 3
            if calculate_angle_gap(self.entry_angle, self.exit_angle) < np.pi / 2:
                d_cita = calculate_angle_gap(self.entry_angle, (self.entry_angle + np.pi/2) % (2*np.pi))
            else:
                Cita = calculate_angle_gap(self.entry_angle, self.exit_angle) / 2
                d_cita = calculate_angle_gap(self.entry_angle, (self.entry_angle + Cita) % (2*np.pi))
            target_x = min(inner_radius + self.length, self.radius) * d_cita
            x_weight = np.exp(-(self.gammar - 1) * (abs(position_x_local - target_x) / target_x))

        else: # Exiting
            target_y = outer_radius - self.width / 2
            d_cita = calculate_angle_gap(self.entry_angle, self.exit_angle)
            target_x = outer_radius * d_cita
            x_weight = np.exp(-self.gammar * (abs(position_x_local - target_x) / target_x))

        y_weight = 1 - np.exp(-abs(target_y - position_y_local))
        def H1(x):
            return np.where(x <= np.pi, 1, 0)
            
        result = y_weight * np.sign(target_y - position_y_local) * x_weight * H1(d_cita) * 4
        return max(-4, min(4, result))

    def calcAccB(self, widthLeft, widthRight, y):
        """Calculates acceleration from road boundaries to keep vehicle within lanes."""
        Wroad1 = widthRight + 3
        Wroad2 = widthLeft
        syRight = self.radius - 5 - Wroad1
        syLeft = Wroad2 - self.radius - 5
        
        alphaLatLeft = np.exp(-syLeft / 0.2) if syLeft > 0 else 1 - syLeft / 0.2
        alphaLatRight = np.exp(-syRight / 0.2) if syRight > 0 else 1 - syRight / 0.2
        
        accLatB0 = 6 * (min(alphaLatRight, 6) - min(alphaLatLeft, 6))
        accLatB = accLatB0 * (0.2 + 0.8 * self.tangential_speed / desired_speed)
        return max(-6, min(6, accLatB))

    def calculate_effective_radius(self):
        """Determines the effective road boundaries based on the vehicle's position."""
        if calculate_angle_gap(self.entry_angle, self.angle) < np.math.asin(10 / outer_radius):
            effective_inner_radius = outer_radius - 15
            effective_outer_radius = outer_radius
        elif calculate_angle_gap(self.angle, self.exit_angle) < np.math.asin(30 / outer_radius):
            effective_inner_radius = outer_radius - 10
            effective_outer_radius = outer_radius
        else:
            effective_inner_radius = inner_radius
            effective_outer_radius = outer_radius
        return effective_inner_radius, effective_outer_radius
        
    def __str__(self):
        return f"Vehicle {self.idx}: Angle={np.degrees(self.angle):.2f}°, R={self.radius:.2f}m, V_tan={self.tangential_speed:.2f}m/s"

def find_front_vehicle1(vehicle, vehicles):
    """Finds the closest vehicle directly ahead when entering the roundabout."""
    front_vehicles = []
    for other in vehicles:
        if other.idx != vehicle.idx and other.radius < vehicle.radius:
             if (vehicle.angle == other.angle and other.radius > outer_radius) or \
                (calculate_angle_gap(vehicle.angle, other.angle) < np.pi/18 and outer_radius-5 < other.radius < outer_radius):
                front_vehicles.append(other)
    
    if not front_vehicles:
        return None
        
    # Return the vehicle with the smallest radial distance
    return min(front_vehicles, key=lambda v: vehicle.radius - v.radius)

def find_front_vehicle(vehicle, vehicles):
    """Finds the leading vehicle in the roundabout based on angle and proximity."""
    front_vehicles = []
    for other in vehicles:
        if other.idx == vehicle.idx or other.radius > outer_radius:
            continue
        
        angle_diff = calculate_angle_gap(vehicle.angle, other.angle)
        arc_length = min(vehicle.radius * angle_diff, other.radius * angle_diff)
        radius_diff = abs(vehicle.radius - other.radius)
        angle_to_exit = calculate_angle_gap(vehicle.angle, vehicle.exit_angle)

        if 0 < arc_length <= 100 and radius_diff < 5 and angle_diff < angle_to_exit:
            front_vehicles.append(other)

    if not front_vehicles:
        return None

    # Select the best leader based on minimizing deceleration
    best_vehicle = min(front_vehicles, key=lambda v:
        idm_int(vehicle.tangential_speed, v.tangential_speed, 
                min(vehicle.radius, v.radius) * calculate_angle_gap(vehicle.angle, v.angle) - vehicle.length, 
                vehicle.radius - v.radius)
    )

    min_decel = idm_int(vehicle.tangential_speed, best_vehicle.tangential_speed, 
                          min(vehicle.radius, best_vehicle.radius) * calculate_angle_gap(vehicle.angle, best_vehicle.angle) - vehicle.length, 
                          vehicle.radius - best_vehicle.radius)

    # Special condition when close to exit to force yielding
    angle_diff2 = calculate_angle_gap(vehicle.angle, vehicle.exit_angle)
    if min_decel > -2 and angle_diff2 < np.math.asin(30 / outer_radius):
        return 1 # Returns a flag instead of a vehicle object
    
    return best_vehicle

def find_follower(vehicle, vehicles):
    """Finds the following vehicle in the roundabout."""
    followers = []
    for other in vehicles:
        if other.idx == vehicle.idx:
            continue

        angle_diff = calculate_angle_gap(other.angle, vehicle.angle)
        if not (0 < angle_diff < np.pi):
            continue
            
        arc_length = min(vehicle.radius * angle_diff, other.radius * angle_diff)
        radius_diff = abs(vehicle.radius - other.radius)
        angle_to_exit_other = calculate_angle_gap(other.angle, vehicle.exit_angle)

        if 0 < arc_length <= 100 and radius_diff < vehicle_length and angle_diff > angle_to_exit_other:
            followers.append(other)

    if not followers:
        return None
        
    # Select the best follower based on minimizing their required deceleration
    return min(followers, key=lambda v:
        idm_int(v.tangential_speed, vehicle.tangential_speed, 
                min(vehicle.radius, v.radius) * calculate_angle_gap(v.angle, vehicle.angle) - vehicle.length, 
                v.radius - vehicle.radius)
    )

### 3. Simulation Setup and Execution
This cell initializes the roundabout's entry/exit points, creates a pool of vehicles, and runs the main simulation loop. Vehicle states are recorded at each time step.

In [20]:
import numpy as np

# Parameters of the circular roundabout
inner_radius = 46  # Inner radius of the roundabout
outer_radius = 84  # Outer radius of the roundabout
vehicle_width = 1.8  # Vehicle width (m)
vehicle_length = 4.5  # Vehicle length (m)
desired_speed = 15 # Desired speed (m/s)
s0 =4 # Minimum safe distance (m)
max_acceleration = 3 # Maximum acceleration (m/s²)
comfortable_deceleration = 4  # Comfortable deceleration (m/s²)
T = 1
# Time parameters
dt = 0.1  # Time step (s)
outer_radius1 = 64  # Outer radius of the roundabout
total_time =100  # Total simulation time (s)
# Entry and exit angle calculations based on your given formula

entry_angles = [
    np.arctan2(np.sqrt(outer_radius**2 - (-3/1)**2), -3/1) + 2*np.pi,  # Entry 1
    np.arctan2(-np.sqrt(outer_radius**2 - (3/1)**2), 3/1) + 2*np.pi,   # Entry 2
    np.arctan2(-3/1, -np.sqrt(outer_radius**2 - (-3/1)**2)) + 2*np.pi,   # Entry 3
    np.arctan2(3/1, np.sqrt(outer_radius**2 - (3/1)**2)) + 2*np.pi     # Entry 4
]

exit_angles = [
    np.arctan2(np.sqrt(outer_radius**2 - (3/1)**2), 3/1) + 2*np.pi,   # Exit 1
    np.arctan2(-np.sqrt(outer_radius**2 - (-3/1)**2), -3/1) + 2*np.pi,  # Exit 2
    np.arctan2(-3/1, np.sqrt(outer_radius**2 - (3/1)**2)) + 2*np.pi,    # Exit 3
    np.arctan2(3/1, -np.sqrt(outer_radius**2 - (-3/1)**2)) + 2*np.pi    # Exit 4
]

# # Generate new entry and exit angles (Insert two new angles spaced by pi/6 between each entry and exit)
new_entry_angles = []
new_exit_angles = []

for i in range(4):
    # Add the entry angles, each separated by pi/6
    new_entry_angles.append((entry_angles[i], f"Entry {i + 1}"))
    new_entry_angles.append((entry_angles[i] + np.pi/6, f"Entry {i + 1}_next_1"))  # Add pi/6
    new_entry_angles.append((entry_angles[i] + 2*np.pi/6, f"Entry {i + 1}_next_2"))  # Add 2pi/6 (another pi/6)
    
for i in range(4):
    # Add the exit angles, each separated by pi/6
    new_exit_angles.append((exit_angles[i], f"Exit {i + 1}"))
    new_exit_angles.append((exit_angles[i] + np.pi/6, f"Exit {i + 1}_next_1"))  # Add pi/6
    new_exit_angles.append((exit_angles[i] + 2*np.pi/6, f"Exit {i + 1}_next_2"))  # Add 2pi/6 (another pi/6)



# Apply % 2π to each angle first to ensure all angles are within 0 to 2π
adjusted_angles_entry = [(angle[0] % (2 * np.pi), angle[1]) for angle in new_entry_angles]
adjusted_angles_exit = [(angle[0] % (2 * np.pi), angle[1]) for angle in new_exit_angles]

# Sort the angles in ascending order
adjusted_angles_entry1 = sorted(adjusted_angles_entry, key=lambda x: x[0])
adjusted_angles_exit1 = sorted(adjusted_angles_exit, key=lambda x: x[0])

# Reassign sorted angles back to entry and exit angles
entry_angles_sorted = [angle[0] for angle in adjusted_angles_entry1 if "Entry" in angle[1]]
exit_angles_sorted = [angle[0] for angle in adjusted_angles_exit1 if "Exit" in angle[1]]

entry_angles = entry_angles_sorted
exit_angles = exit_angles_sorted
# Initialize vehicles
vehicles = []
vehicle_positions = {}
vehicle_counter = 0
num_vehicles =100

# Pre-initialize vehicles and assign random entry and exit points
for i in range(num_vehicles):
    # Randomly assign entry and exit
#     entry_idx = np.random.randint(0, 4)
    entry_idx = i % 12
#     exit_idx = (i+4) % 8
    exit_idx = np.random.randint(0, 12)
    gammar = 3
#     gammar = np.random.uniform(2, 4)
    entry_angle = entry_angles[entry_idx]
    exit_angle = exit_angles[exit_idx]
    
    paused_vehicle = Vehicle(i, 0, 999, entry_angle=entry_angle, exit_angle=exit_angle)  # Paused at (999, 999)
    paused_vehicle.position_x.append(999)
    paused_vehicle.position_y.append(999)
    paused_vehicle.entry_idx = entry_idx  # Store the entry index
    paused_vehicle.exit_idx = exit_idx    # Store the exit index
    paused_vehicle.gammar = gammar    # Store the exit index
    vehicles.append(paused_vehicle)
    vehicle_positions[f"Veh[{i}]"] = {
        "position_x": [],
        "position_y": [],
        "tangential_speeds": [],
        "radial_speeds": [],
        "exit_idx": [],  # Store the exit index for each vehicle
        "entry_idx": [],        
        "radius": [],
        "angle": [],
        # Store the exit index for each vehicle
    }


# Run simulation
for t in range(int(total_time / dt)):
    flow_rate = 1.8 # 0.8 vehicle/second for the first stage
    spawn_interval = flow_rate
    # Add a new vehicle based on the spawn interval
    if t % int(spawn_interval / dt) == 0:
        for vehicle in vehicles:
            if vehicle.paused:  # Check for a paused vehicle
                # Set vehicle's initial position on the entry lane
                vehicle.angle = entry_angles[vehicle.entry_idx]  # Entry at the appropriate angle
                vehicle.radius = outer_radius+30  # Start outside the roundabout
                vehicle.paused = False  # Activate the vehicle
                vehicle.tangential_speed = 0  # Initial tangential speed
                vehicle.radial_speed = -10  # Initial radial speed towards the roundabout
                vehicle.position_x[-1] = np.cos(vehicle.angle) * vehicle.radius
                vehicle.position_y[-1] = np.sin(vehicle.angle) * vehicle.radius
                break  # Only activate one vehicle per interval
    active_vehicles = [vehicle for vehicle in vehicles if not vehicle.paused]          
    for vehicle in vehicles:
        if vehicle.paused:
            # Keep paused vehicles at (999, 999)
            vehicle.position_x.append(999)
            vehicle.position_y.append(999)
            vehicle.tangential_speeds.append(0)
            vehicle.radial_speeds.append(0)
            vehicle.decide = 999
            vehicle.angle = 0  # Entry at the appropriate angle
            vehicle.radius = 999  # Start outside the roundabout
            vehicle_positions[f"Veh[{vehicle.idx}]"]["position_x"].append(vehicle.position_x[-1])
            vehicle_positions[f"Veh[{vehicle.idx}]"]["position_y"].append(vehicle.position_y[-1])
            vehicle_positions[f"Veh[{vehicle.idx}]"]["tangential_speeds"].append(vehicle.tangential_speed)
            vehicle_positions[f"Veh[{vehicle.idx}]"]["radial_speeds"].append(vehicle.radial_speed)
            vehicle_positions[f"Veh[{vehicle.idx}]"]["exit_idx"].append(vehicle.exit_idx)
            vehicle_positions[f"Veh[{vehicle.idx}]"]["entry_idx"].append(vehicle.entry_idx)
            vehicle_positions[f"Veh[{vehicle.idx}]"]["radius"].append(vehicle.radius)
            vehicle_positions[f"Veh[{vehicle.idx}]"]["angle"].append(vehicle.angle)
            continue


        
        else:

            vehicle.update_decision(entry_angles, exit_angles)
            if vehicle.radius>outer_radius:
                front_vehicle1 = find_front_vehicle1(vehicle, active_vehicles)
                vehicle.update(front_vehicle1,None, dt)
            else:
                # Find front vehicle in the roundabout if necessary
                front_vehicle = find_front_vehicle(vehicle, active_vehicles)
                follower_vehicle = find_follower(vehicle, active_vehicles)
                vehicle.update(front_vehicle,follower_vehicle, dt)
                # 假设 theta1 和 theta 为目标角度和起始角度，vehicle.angle 为车辆的实时角度
                angle_diff = calculate_angle_gap(vehicle.angle, vehicle.exit_angle)
            # 计算目标角度与当前角度之间的差值，确保差值始终是正值
            if  vehicle.decide<0 and (outer_radius+32<vehicle.radius) :  # 允许微小差距
                # 当车辆角度超过目标角度时，暂停车辆
                
                vehicle.paused = True
                vehicle.position_x.append(999)  # 移动到 (999, 999)
                vehicle.position_y.append(999)
                vehicle.tangential_speeds.append(0)
                vehicle.radial_speeds.append(0)
                vehicle.angle = 0  # Entry at the appropriate angle
                vehicle.radius = 999  # Start outside the roundabout
                vehicle_positions[f"Veh[{vehicle.idx}]"]["exit_idx"].append(vehicle.exit_idx)
                vehicle.decide = 999
                # 记录暂停后的车辆信息
                vehicle_positions[f"Veh[{vehicle.idx}]"]["position_x"].append(vehicle.position_x[-1])
                vehicle_positions[f"Veh[{vehicle.idx}]"]["position_y"].append(vehicle.position_y[-1])
                vehicle_positions[f"Veh[{vehicle.idx}]"]["tangential_speeds"].append(vehicle.tangential_speed)
                vehicle_positions[f"Veh[{vehicle.idx}]"]["radial_speeds"].append(vehicle.radial_speed)
                vehicle_positions[f"Veh[{vehicle.idx}]"]["exit_idx"].append(vehicle.exit_idx)
                vehicle_positions[f"Veh[{vehicle.idx}]"]["entry_idx"].append(vehicle.entry_idx)
                vehicle_positions[f"Veh[{vehicle.idx}]"]["radius"].append(vehicle.radius)
                vehicle_positions[f"Veh[{vehicle.idx}]"]["angle"].append(vehicle.angle)
            else:
                # Find front vehicle in the roundabout

                vehicle_positions[f"Veh[{vehicle.idx}]"]["position_x"].append(vehicle.position_x[-1])
                vehicle_positions[f"Veh[{vehicle.idx}]"]["position_y"].append(vehicle.position_y[-1])
                vehicle_positions[f"Veh[{vehicle.idx}]"]["tangential_speeds"].append(vehicle.tangential_speed)
                vehicle_positions[f"Veh[{vehicle.idx}]"]["radial_speeds"].append(vehicle.radial_speed)
                vehicle_positions[f"Veh[{vehicle.idx}]"]["exit_idx"].append(vehicle.exit_idx)
                vehicle_positions[f"Veh[{vehicle.idx}]"]["entry_idx"].append(vehicle.entry_idx)
                vehicle_positions[f"Veh[{vehicle.idx}]"]["radius"].append(vehicle.radius)
                vehicle_positions[f"Veh[{vehicle.idx}]"]["angle"].append(vehicle.angle)
            

### 4. Visualization
This cell animates the simulation results stored in the `vehicle_positions` dictionary. It generates a plot for each time step, showing vehicle positions, orientations, and speeds (represented by color).

In [22]:
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.patches import Rectangle, Wedge
from IPython.display import display, clear_output
import math
from matplotlib.transforms import Affine2D
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.patches import Rectangle, Wedge
from IPython.display import display, clear_output
import math

vehicle_width = 1.6  # Vehicle width (m)
vehicle_length = 4.2  # Vehicle length (m)
# 速度可视化标准化和颜色映射
norm = Normalize(vmin=0, vmax=15)
cmap = plt.get_cmap('rainbow_r')
sm = ScalarMappable(cmap=cmap, norm=norm)
num_points = 12  # 进出点数量
inner_radius = 46  # Inner radius of the roundabout
outer_radius = 84  # Outer radius of the roundabout
outer_radius1 = 81.5  # Outer radius of the roundabout
v_speed1=20
# Entry and exit angles based on your given formula
entry_angles = [
    np.arctan2(np.sqrt(outer_radius**2 - (-6/1)**2), -6/1) + 2*np.pi,  # Entry 1
    np.arctan2(-np.sqrt(outer_radius**2 - (6/1)**2), 6/1) + 2*np.pi,   # Entry 2
    np.arctan2(-6/1, -np.sqrt(outer_radius**2 - (-6/1)**2)) + 2*np.pi,   # Entry 3
    np.arctan2(6/1, np.sqrt(outer_radius**2 - (6/1)**2)) + 2*np.pi     # Entry 4
]

exit_angles = [
    np.arctan2(np.sqrt(outer_radius**2 - (6/1)**2), 6/1) + 2*np.pi,   # Exit 1
    np.arctan2(-np.sqrt(outer_radius**2 - (-6/1)**2), -6/1) + 2*np.pi,  # Exit 2
    np.arctan2(-6/1, np.sqrt(outer_radius**2 - (6/1)**2)) + 2*np.pi,    # Exit 3
    np.arctan2(6/1, -np.sqrt(outer_radius**2 - (-6/1)**2)) + 2*np.pi    # Exit 4
]

# # Generate new entry and exit angles (Insert two new angles spaced by pi/6 between each entry and exit)
new_entry_angles = []
new_exit_angles = []

for i in range(4):
    # Add the entry angles, each separated by pi/6
    new_entry_angles.append((entry_angles[i], f"Entry {i + 1}"))
    new_entry_angles.append((entry_angles[i] + np.pi/6, f"Entry {i + 1}_next_1"))  # Add pi/6
    new_entry_angles.append((entry_angles[i] + 2*np.pi/6, f"Entry {i + 1}_next_2"))  # Add 2pi/6 (another pi/6)
    
for i in range(4):
    # Add the exit angles, each separated by pi/6
    new_exit_angles.append((exit_angles[i], f"Exit {i + 1}"))
    new_exit_angles.append((exit_angles[i] + np.pi/6, f"Exit {i + 1}_next_1"))  # Add pi/6
    new_exit_angles.append((exit_angles[i] + 2*np.pi/6, f"Exit {i + 1}_next_2"))  # Add 2pi/6 (another pi/6)



# Apply % 2π to each angle first to ensure all angles are within 0 to 2π
adjusted_angles_entry = [(angle[0] % (2 * np.pi), angle[1]) for angle in new_entry_angles]
adjusted_angles_exit = [(angle[0] % (2 * np.pi), angle[1]) for angle in new_exit_angles]

# Sort the angles in ascending order
adjusted_angles_entry1 = sorted(adjusted_angles_entry, key=lambda x: x[0])
adjusted_angles_exit1 = sorted(adjusted_angles_exit, key=lambda x: x[0])

# Reassign sorted angles back to entry and exit angles
entry_angles_sorted = [angle[0] for angle in adjusted_angles_entry1 if "Entry" in angle[1]]
exit_angles_sorted = [angle[0] for angle in adjusted_angles_exit1 if "Exit" in angle[1]]

entry_angles = entry_angles_sorted
exit_angles = exit_angles_sorted
# Plotting the roundabout and the entry/exit points
def calculate_position(angle, radius):
    x = radius * np.cos(angle)
    y = radius * np.sin(angle)
    return x, y

def cart_to_polar(x, y):
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan2(y, x)
    return r, theta

def adjust_text_position(x, y, offset_radius=12):
    r, theta = cart_to_polar(x, y)
    text_x = x + offset_radius * np.cos(theta)
    text_y = y + offset_radius * np.sin(theta)
    return text_x, text_y

def calculate_rotation(theta):
    rotation = np.degrees(theta)
    return rotation

def visualize_entry_exit_points(ax):
    for i in range(len(entry_angles)):
        # Calculate entry point position
        entry_x, entry_y = calculate_position(entry_angles[i], outer_radius)
        # Adjust entry point text position and calculate rotation
        entry_text_x, entry_text_y = adjust_text_position(entry_x, entry_y, offset_radius=15)
        entry_rotation = calculate_rotation(np.arctan2(entry_y, entry_x))
        # Plot entry points and their labels
        ax.plot(entry_x, entry_y, 'gs', markersize=10, label=f"Entry {i + 1}" if i == 0 else "", alpha=0.7)
        ax.text(entry_text_x, entry_text_y, f"Entry {i + 1}", color='green', ha='center', va='bottom', 
                fontsize=12, rotation=entry_rotation, rotation_mode='anchor')
        
        # Calculate exit point position
        exit_x, exit_y = calculate_position(exit_angles[i], outer_radius)
        # Adjust exit point text position and calculate rotation
        exit_text_x, exit_text_y = adjust_text_position(exit_x, exit_y, offset_radius=15)
        exit_rotation = calculate_rotation(np.arctan2(exit_y, exit_x))
        # Plot exit points and their labels
        ax.plot(exit_x, exit_y, 'rs', markersize=10, label=f"Exit {i + 1}" if i == 0 else "", alpha=0.7)
        ax.text(exit_text_x, exit_text_y, f"Exit {i + 1}", color='red', ha='center', va='bottom', 
                fontsize=12, rotation=exit_rotation, rotation_mode='anchor')

        # Draw parallel lines for the entry/exit points
        ax.plot([entry_x, entry_x + 30 * np.cos(entry_angles[i])],
                [entry_y, entry_y + 30 * np.sin(entry_angles[i])], 'k--', linestyle='dashed', linewidth=2)
        ax.plot([exit_x, exit_x + 30 * np.cos(exit_angles[i])],
                [exit_y, exit_y + 30 * np.sin(exit_angles[i])], 'k--', linestyle='dashed', linewidth=2)

fig, ax = plt.subplots(figsize=(10, 10))
plt.rcParams.update({'font.size': 20})
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['axes.unicode_minus'] = False
np.set_printoptions(suppress=True)
plt.rcParams.update({'text.usetex': False, 'font.family': 'serif', 'font.serif': 'Times New Roman', 'mathtext.fontset': 'cm'})


# Visualization loop
for j in range(0,480):  # Loop through simulation steps
    t=j*v_speed1
    ax.clear()

    # Set plot boundaries and properties
    ax.set_xlim(-outer_radius - 50, outer_radius + 50)
    ax.set_ylim(-outer_radius - 50, outer_radius + 50)
    ax.set_aspect('equal')
    ax.set_title("Real-Time Vehicle Positions in a Discretized Roundabout")
    ax.set_xlabel("X (m)")
    ax.set_ylabel("Y (m)")

    # Fill the area between the inner and outer circles
    roundabout_region = Wedge((0, 0), outer_radius + 2, 0, 360, width=outer_radius - inner_radius + 4, color='lightgray', alpha=0.5)
    ax.add_patch(roundabout_region)
    visualize_entry_exit_points(ax)
    # Draw roundabout boundaries
    inner_circle = plt.Circle((0, 0), inner_radius, fill=False, color='black', linestyle='dashed', linewidth=3)
    outer_circle = plt.Circle((0, 0), outer_radius, fill=False, color='black', linestyle='dashed', linewidth=3)
    outer_circle1 = plt.Circle((0, 0), outer_radius-5, fill=False, color='white', linestyle='dashed', linewidth=2, alpha=0.5)
    ax.add_artist(inner_circle)
    ax.add_artist(outer_circle)
    ax.add_artist(outer_circle1)
    # Plot each vehicle
    for i in range(num_vehicles):
        x = vehicle_positions[f"Veh[{i}]"]["position_x"][t]
        y = vehicle_positions[f"Veh[{i}]"]["position_y"][t]
        vx = vehicle_positions[f"Veh[{i}]"]["tangential_speeds"][t]
        vy = vehicle_positions[f"Veh[{i}]"]["radial_speeds"][t]
        
        speed = math.sqrt(vx**2 + vy**2)
        color = sm.to_rgba(speed)

        epsilon = 1e-6 

        # Calculate tangent and radial angles
        tangent_angle1 = np.arctan2(vx, vy)  # Assuming circular path centered at origin
        tangent_angle= np.arctan2(y, x)  # Assuming circular path centered at origin
        
#         if -10<x <10 and y<0:        
#             orientation_angle = tangent_angle1+ np.pi / 2    # 加 90 度
#         else:

        distances = np.sqrt(x**2 + y**2)
        if distances>outer_radius:
   
#             if  np.arctan2(x, y)<=epsilon:
            orientation_angle = np.arctan2(y,x)# 加 90 度

#             else:
#                 orientation_angle = np.arctan2(y, x)# 加 90 度           
        else:

            if vx<=epsilon:
                orientation_angle = tangent_angle+ np.pi / 2 # 加 90 度

            else:
                orientation_angle = tangent_angle +tangent_angle1 # 加 90 度


    #         visualize_entry_exit_points(ax)

#      # Plot vehicle as a rectangle
        rect = Rectangle(
            (x - vehicle_length / 2, y - vehicle_width / 2),  # Bottom-left corner of the rectangle
            vehicle_length,  # Width of the vehicle
            vehicle_width,   # Height of the vehicl # Convert radians to degrees for the angle
            color=color,     # Use the calculated color based on speed
            alpha=0.7,         # Full opacity
          # Ensure that transformation is applied in the correct coordinate system
        )

        rotate_transform = Affine2D().translate(-x, -y)  # Move the origin to (x, y)
        rotate_transform += Affine2D().rotate_deg(np.degrees(orientation_angle))  # Apply the rotation
        rotate_transform += Affine2D().translate(x, y)  # Translate it back to (x, y)

        # Apply the transform to the rectangle
        rect.set_transform(rotate_transform + ax.transData)

        # Add the rotated rectangle to the plot
        ax.add_patch(rect)

        # Set equal aspect ratio for proper scaling
        ax.set_aspect('equal')
#                 ax.add_patch(rect)  # Add the rectangle to the plot
        if -outer_radius - 30 < x < outer_radius + 30:
            exit_id = vehicle_positions[f"Veh[{i}]"]["exit_idx"][t]  # Access the exit index for the current vehicle at time step t
            entry_idx = vehicle_positions[f"Veh[{i}]"]["entry_idx"][t]  # Access the entry index for the current vehicle at time step t

            # Format the text to display both entry and exit IDs
            entry_exit_text = f"{entry_idx+1},{exit_id+1}"

#             Display the formatted text at the vehicle's position
            ax.text(x, y, entry_exit_text, color='black', ha='center', va='center', fontsize=10)

    ax.set_title(f'T={t/10:.1f}s', fontname='Times New Roman', fontsize=15)
            
    
    # Display the updated figure
    display(fig)
    plt.pause(0.0001)
    clear_output(wait=True)

plt.show()

IndexError: list index out of range